# Train the Model using datas

In [16]:
%%capture
%run ../config/config.ipynb

In [7]:
from torch.optim import Adam
from datetime import datetime
import torch 
from tqdm import tqdm

In [8]:
%%capture
%run ./data-loader.ipynb

In [9]:
%%capture
%run ./Transformer.ipynb

In [17]:
# Prepare the model 
model = Transformer(
    src_pad_token=src_pad_token, 
    trg_pad_token=trg_pad_token, 
    trg_sos_token=trg_sos_token, 
    enc_voc_size=enc_voc_size, 
    dec_voc_size=dec_voc_size, 
    n_head=n_head, 
    max_len=max_len, 
    d_model=d_model, 
    ffn_hidden=ffn_hidden, 
    n_layers=n_layers, 
    drop_prob=drop_prob, 
    device=device).to(device)

model.train()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'model parameter #: {count_parameters(model)}')


model parameter #: 84332346


In [9]:
# Setup optimizer 
optimizer = Adam(params=model.parameters(), lr=init_lr, weight_decay=weight_decay)

# Setup loss function for training
loss_func = nn.CrossEntropyLoss(ignore_index=src_pad_token)


In [10]:
def train_epoch(epoch_num): 
    train_epoch_loss = 0 

    for step, (kr_tokenized, en_tokenized) in tqdm(enumerate(train_dataloader)): 
        optimizer.zero_grad()

        kr_tokenized = kr_tokenized.to(device)
        en_tokenized = en_tokenized.to(device)

        out = model(kr_tokenized, en_tokenized[:, :-1])

        # remove sos token from en_tokenized when calculating loss because out will not include sos token. 
        en_tokenized = en_tokenized[:, 1:].contiguous().view(-1)

        out = out.contiguous().view(-1, out.shape[-1])

        loss = loss_func(out, en_tokenized.type(torch.LongTensor))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        train_epoch_loss += loss.item()

    train_step_loss = train_epoch_loss / (step+1)
    # After training epoch, do evaluation 

    return train_step_loss
    



IndentationError: unexpected indent (2647276441.py, line 42)

In [ ]:
# evaluate the model 
def evaluate(): 
    model.eval()
    test_epoch_loss = 0 
    test_bleu_loss = 0
    
    with torch.no_grad(): 
        for step, (kr_tokenized, en_tokenized) in tqdm(enumerate(test_dataloader)): 
            kr_tokenized = kr_tokenized.to(device)
            en_tokenized = en_tokenized.to(device)

            out = model(kr_tokenized, en_tokenized[:, :-1])

            # remove sos token from en_tokenized when calculating loss because out will not include sos token. 
            en_tokenized = en_tokenized[:, 1:].contiguous().view(-1)
    
            out = out.contiguous().view(-1, out.shape[-1])
            loss = loss_func(out, en_tokenized.type(torch.LongTensor))
            test_epoch_loss += loss.item()

            # calcuate the bleu 
            # TODO
    return test_step_loss

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
best_vloss = 100_000_000

for epoch in range(epochs):
    train_loss = train_epoch(epoch)
    test_loss = evaluate()

    print(f'Epoch {epoch}: Train Loss {train_loss}, Test Loss {test_loss}')

    if test_loss < best_vloss:
        best_vloss = avg_vloss 
        model_path = f'models/model_{timestamp}_{epoch}' 
        torch.save(model.state_dict(), model_path)  